## <center> DLD image processing </center>
### <center> By Dr. Joseph Vas

In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sys, os, glob

In [2]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


Gain referencing

In [3]:
def compute_gain_reference(path):
    
    k =int((1620-1024)/2) #this needs to be changed later
    
    extension = '*.tif'
    folder_check = path+'/'+extension
    files = glob.glob(folder_check)
    
    calibration_images = []
    
    for file in files:
        n_num = np.array(Image.open(file)).shape[0]
        calibration_images.append(np.array(Image.open(file))[k:n_num-k,k:n_num-k])
    num_images = len(calibration_images)

    # Get image size from the first calibration image
    image_height, image_width = calibration_images[0].shape

    # Initialize an array to store the average pixel values
    average_pixel_values = np.zeros((image_height, image_width), dtype=np.float32)

    # Compute the average pixel values for each image
    for image in calibration_images:
        average_pixel_values += image.astype(np.float32) / num_images

    # Normalize the average pixel values by dividing by a reference level
    reference_level = np.max(average_pixel_values)  # Use the maximum pixel value as the reference level
    normalized_average_pixel_values = average_pixel_values / reference_level

    return normalized_average_pixel_values

def compute_processed_image(raw_image, dark_reference, gain_reference):
    # Subtract dark reference
    processed_image = raw_image #- dark_reference

    # Apply gain correction
    processed_image = np.divide(processed_image, gain_reference)

    return processed_image



In [9]:
path = '/Users/josephvas/Dropbox/ER-C/Development work/DLD/2023_06_07/Intensity_series/'
gain_reference = compute_gain_reference(path)
#print(gain_reference)

In [112]:
k =int((1620-1024)/2)

path2 = '/Users/josephvas/Dropbox/ER-C/Projects/DLD/Holo/2023_06_07/holograms/'
n_num = np.array(Image.open(path2+'biprism_10000_C2_30.672.tif')).shape[0]
raw_image = np.array(Image.open(path2+'biprism_10000_C2_30.672.tif'))[k:n_num-k,k:n_num-k]
dark_reference = np.array(Image.open(path+'4e6.tif'))[k:n_num-k,k:n_num-k]

In [113]:
processed_image = compute_processed_image(raw_image, dark_reference, gain_reference)
image_fft = abs(np.fft.fftshift(np.fft.fft2(processed_image)))

processed_image = raw_image
image_fft = abs(np.fft.fftshift(np.fft.fft2(processed_image)))

In [114]:
vmin_mul = 0
vmax_mul = 2.5

fig, ax = plt.subplots(nrows = 1, ncols= 2)

ax[0].imshow(processed_image, cmap='gray', \
                 vmin=vmin_mul*np.mean(processed_image), vmax=vmax_mul*np.mean(processed_image))

ax[1].imshow((image_fft), cmap='gray')